In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import os
from google.colab import drive
drive.mount('/content/gdrive')

df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/heart_cleveland_upload.csv",sep=';')

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import pandas_profiling as pp


In [3]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,40,1,3,152,223,0,0,181,0,0.0,0,0,2,1
293,39,1,3,118,219,0,0,140,0,1.2,1,0,2,1
294,35,1,3,120,198,0,0,130,1,1.6,1,0,2,1
295,35,0,3,138,183,0,0,182,0,1.4,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        297 non-null    int64  
 1   sex        297 non-null    int64  
 2   cp         297 non-null    int64  
 3   trestbps   297 non-null    int64  
 4   chol       297 non-null    int64  
 5   fbs        297 non-null    int64  
 6   restecg    297 non-null    int64  
 7   thalach    297 non-null    int64  
 8   exang      297 non-null    int64  
 9   oldpeak    297 non-null    float64
 10  slope      297 non-null    int64  
 11  ca         297 non-null    int64  
 12  thal       297 non-null    int64  
 13  condition  297 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 32.6 KB


In [5]:
target_name = 'condition'
data_target = df[target_name]
data = df.drop([target_name], axis=1)

In [6]:
train, test, target, target_test = train_test_split(data, data_target, test_size=0.3, random_state=0)

In [7]:
#'age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','ca','thal','condition'
#'sex', 'fbs','cp', 'restecg','exang','thal','age' , 'trestbps'
#np.where(train.dtypes != np.float)[0]
categorical_features_indices = ['sex', 'fbs','cp', 'restecg','exang','thal','age' , 'trestbps']
accuracy =[]
model_names =[]

In [8]:
categorical_features_indices 

['sex', 'fbs', 'cp', 'restecg', 'exang', 'thal', 'age', 'trestbps']

In [9]:
pip install catboost --upgrade --no-deps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from catboost import CatBoostClassifier, Pool

In [11]:
model = CatBoostClassifier(verbose=False,random_state=0)

model.fit(train, target,cat_features=categorical_features_indices,eval_set=(test, target_test))
y_pred = model.predict(test)
accuracy.append(round(accuracy_score(target_test, y_pred),4))

model_names = ['Catboost_default']
result_df5 = pd.DataFrame({'Accuracy':accuracy}, index=model_names)
result_df5

,Accuracy
Catboost_default,0.8889


In [ ]:
!pip install --quiet optuna

     |████████████████████████████████| 348 kB 5.3 MB/s 
     |████████████████████████████████| 209 kB 61.6 MB/s 
     |████████████████████████████████| 81 kB 9.0 MB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
     |████████████████████████████████| 112 kB 72.9 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 147 kB 55.9 MB/s 


In [ ]:
import optuna

optuna.__version__

'3.0.3'

In [ ]:
def objective(trial):
    
    
    X_train, X_test, y_train, y_test = train_test_split(data, data_target, test_size=0.3, random_state=42)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    cat_cls = CatBoostClassifier(**param)

    cat_cls.fit(X_train, y_train, eval_set=[(X_test, y_test)], cat_features=categorical_features_indices,verbose=0, early_stopping_rounds=100)

    preds = cat_cls.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2022-11-10 10:17:54,204] A new study created in memory with name: no-name-4cbb477a-d1e8-4d41-8f29-8b0876d88e9f
[I 2022-11-10 10:17:55,308] Trial 0 finished with value: 0.8 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.02335035265312914, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.9406972571682}. Best is trial 0 with value: 0.8.
[I 2022-11-10 10:17:55,880] Trial 1 finished with value: 0.8333333333333334 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.0760078446604263, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 1 with value: 0.8333333333333334.
[I 2022-11-10 10:17:57,007] Trial 2 finished with value: 0.8 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.010994893958341849, 'depth': 4, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.7868195159893278}. Best is trial 1 with value: 0.8333333333333334.
[I 2022-11-10 10:17:57,710] 

Number of finished trials: 50
Best trial:
  Value: 0.8777777777777778
  Params: 
    objective: CrossEntropy
    colsample_bylevel: 0.055468477840894814
    depth: 5
    boosting_type: Plain
    bootstrap_type: Bernoulli
    subsample: 0.41687168448921785
